# Set environment

In [1]:
source("config_deseq.R")
source("util_deseq.R")

Loading tidyverse: ggplot2
Loading tidyverse: tibble
Loading tidyverse: tidyr
Loading tidyverse: readr
Loading tidyverse: purrr
Loading tidyverse: dplyr
Conflicts with tidy packages ---------------------------------------------------
filter(): dplyr, stats
lag():    dplyr, stats

Attaching package: ‘foreach’

The following objects are masked from ‘package:purrr’:

    accumulate, when

Loading required package: S4Vectors
Loading required package: stats4
Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following o

# Import data

In [2]:
# import RData (annomapres0, annogenecnts0)
#attach(file.path(OUTDIR, "HTS-GroupB-Annotated-STAR-counts.RData"))
attach(file.path(OUTDIR, "HTS-GroupAll-Annotated-STAR-counts.RData"))

Prepare columnData DataFrame and countData (matrix object)
- columnData --- metadata
- countData  --- count matrix

In [3]:
tmp <- annomapres0
table(tmp$Media)
table(tmp$Strain)


 TC YPD 
 24  24 


  H99 mar1d 
   24    24 

In [4]:
# columnData --- metadata
columnData <- annomapres0 %>% as.data.frame %>% mutate(Group = paste0(Media, Strain))
rownames(columnData) <- columnData[["Label"]]

# orgnaize the factor levels
columnData$Media  <- factor(columnData$Media,  levels = c("YPD", "TC"))
columnData$Strain <- factor(columnData$Strain, levels = c("H99", "mar1d"))
columnData$Group  <- factor(columnData$Group,  levels = c("YPDH99", "YPDmar1d", "TCH99", "TCmar1d"))

head(columnData[, c("Label", "Strain", "Media", "Group")], 3)

,Label,Strain,Media,Group
A_1,A_1,H99,YPD,YPDH99
A_13,A_13,H99,TC,TCH99
A_14,A_14,H99,TC,TCH99


remore data from group F

In [5]:
startsWith(x = c("fja", "bja"), prefix = c("f", "b"))

[1] TRUE TRUE

In [6]:
columnData_rmF <- columnData %>% filter(!(Label %in% c("F_27", "F_16")))
rownames(columnData_rmF) <- columnData_rmF[["Label"]]
columnData_rmF$Label
head(columnData_rmF)

[1] "A_1"  "A_13" "A_14" "A_15" "A_2"  "A_3"  "B_16" "B_26" "B_27" "B_38"
[11] "B_4"  "B_40" "C_10" "C_11" "C_21" "C_22" "C_23" "C_9"  "D_12" "D_24"
[21] "D_35" "D_36" "D_45" "D_47" "E_1"  "E_13" "E_14" "E_15" "E_2"  "E_3" 
[31] "F_26" "F_38" "F_4"  "F_40" "G_10" "G_11" "G_21" "G_22" "G_23" "G_9" 
[41] "H_12" "H_24" "H_35" "H_36" "H_45" "H_47"

,Label,Strain,Media,experiment_person,prop.gene,prop.nofeat,prop.unique,depth,Group
A_1,A_1,H99,YPD,expA,0.6151899,0.03635624,0.6515461,3109312,YPDH99
A_13,A_13,H99,TC,expA,0.6476637,0.05793771,0.7056015,2436038,TCH99
A_14,A_14,H99,TC,expA,0.6270156,0.05659609,0.6836117,2802332,TCH99
A_15,A_15,H99,TC,expA,0.6237729,0.06518918,0.6889621,2449520,TCH99
A_2,A_2,H99,YPD,expA,0.6145219,0.04734133,0.6618632,2809212,YPDH99
A_3,A_3,H99,YPD,expA,0.6319960,0.04088589,0.6728819,2688939,YPDH99


In [7]:
# countData  --- count matrix
annogenecnts0 %>%
    dplyr::select(as.character(c("gene", columnData[["Label"]]))) %>%
    as.data.frame %>%
    column_to_rownames("gene") %>%
    as.matrix ->
    countData

head(countData, 3)

,A_1,A_13,A_14,A_15,A_2,A_3,B_16,B_26,B_27,B_38,⋯,G_21,G_22,G_23,G_9,H_12,H_24,H_35,H_36,H_45,H_47
CNAG_04548,0,0,0,0,2,0,0,0,0,1,⋯,0,2,0,0,0,1,1,4,1,0
CNAG_07303,0,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
CNAG_07304,1,1,3,3,1,3,17,2,7,15,⋯,43,29,23,21,20,25,3,19,13,17


remove gene with no reads

In [8]:
countData_rm0 <- countData[rowSums(countData) != 0, ]
head(cbind(countData_rm0, rowSums(countData_rm0)))

,A_1,A_13,A_14,A_15,A_2,A_3,B_16,B_26,B_27,B_38,⋯,G_22,G_23,G_9,H_12,H_24,H_35,H_36,H_45,H_47,
CNAG_04548,0,0,0,0,2,0,0,0,0,1,⋯,2,0,0,0,1,1,4,1,0,48
CNAG_07304,1,1,3,3,1,3,17,2,7,15,⋯,29,23,21,20,25,3,19,13,17,856
CNAG_07305,0,0,0,0,0,0,1,0,0,1,⋯,1,1,2,3,3,0,1,2,1,65
CNAG_00002,10,21,18,23,14,12,92,11,98,173,⋯,135,59,55,55,105,18,86,44,64,4546
CNAG_00003,3,4,7,8,5,3,44,8,26,45,⋯,64,27,39,60,58,9,61,26,32,1947
CNAG_00004,9,37,31,57,12,18,215,22,108,299,⋯,308,115,114,192,239,44,187,119,189,9176


remove samples F16 F27

In [9]:
idx <- colnames(countData_rm0) %in% c("F_27", "F_16")
countData_rm0F <- countData_rm0[, !idx]
colnames(countData_rm0F)

[1] "A_1"  "A_13" "A_14" "A_15" "A_2"  "A_3"  "B_16" "B_26" "B_27" "B_38"
[11] "B_4"  "B_40" "C_10" "C_11" "C_21" "C_22" "C_23" "C_9"  "D_12" "D_24"
[21] "D_35" "D_36" "D_45" "D_47" "E_1"  "E_13" "E_14" "E_15" "E_2"  "E_3" 
[31] "F_26" "F_38" "F_4"  "F_40" "G_10" "G_11" "G_21" "G_22" "G_23" "G_9" 
[41] "H_12" "H_24" "H_35" "H_36" "H_45" "H_47"

Explicitly checking to make sure the column names of count matrix are the same as the rownames of the metadata.

In [10]:
identical(rownames(columnData_rmF), colnames(countData_rm0F))

[1] TRUE

# Make DESeq object on the basis of the counts


In [11]:
dds <- DESeqDataSetFromMatrix(
    countData_rm0F,       # Count matrix
    columnData_rmF,       # metadata
    ~ Group)              # design formula
dds <- DESeq(dds)

estimating size factors
estimating dispersions
gene-wise dispersion estimates
mean-dispersion relationship
final dispersion estimates
fitting model and testing


In [12]:
mcols(dds) %>% as.data.frame

baseMean,baseVar,allZero,dispGeneEst,dispFit,dispersion,dispIter,dispOutlier,dispMAP,Intercept,⋯,WaldStatistic_Group_TCmar1d_vs_YPDH99,WaldPvalue_Intercept,WaldPvalue_Group_YPDmar1d_vs_YPDH99,WaldPvalue_Group_TCH99_vs_YPDH99,WaldPvalue_Group_TCmar1d_vs_YPDH99,betaConv,betaIter,deviance,maxCooks,replace
1.0626700,9.835024e+00,FALSE,1.5863901754,0.41388081,1.224658301,10,FALSE,1.224658301,1.4105150,⋯,-2.6496548,2.416627e-02,5.334178e-03,7.432865e-03,8.057406e-03,TRUE,9,114.3839,0.617382439,FALSE
12.5089065,2.879797e+01,FALSE,0.0000000100,0.05292274,0.016405112,9,FALSE,0.016405112,3.0115504,⋯,5.0488887,5.549691e-58,3.136322e-02,4.755984e-03,4.443876e-07,TRUE,4,241.3823,0.060443892,FALSE
0.8302378,6.461376e-01,FALSE,0.0000000100,0.52431592,0.151033165,8,FALSE,0.151033165,-0.7340897,⋯,1.8612181,1.772083e-01,1.841371e-01,5.051190e-01,6.271339e-02,TRUE,7,113.3822,0.050643784,FALSE
72.3637545,5.884776e+02,FALSE,0.0026483558,0.02520417,0.005844461,10,FALSE,0.005844461,6.2010722,⋯,-2.1307795,0.000000e+00,6.727793e-12,6.391828e-05,3.310731e-02,TRUE,3,327.2186,0.028326730,FALSE
29.7699768,9.372741e+01,FALSE,0.0540871337,0.03349234,0.051506271,10,FALSE,0.051506271,4.5462736,⋯,1.8752707,7.035835e-172,3.357855e-01,3.402613e-03,6.075549e-02,TRUE,3,327.1633,0.085382219,FALSE
138.3470836,1.823147e+03,FALSE,0.0107551956,0.02244134,0.011720495,9,FALSE,0.011720495,6.5137437,⋯,8.1052188,0.000000e+00,5.997082e-03,2.163003e-22,5.265069e-16,TRUE,3,395.1753,0.067478413,FALSE
52.4705134,7.730441e+02,FALSE,0.0168951152,0.02740042,0.018189811,9,FALSE,0.018189811,5.6026606,⋯,2.9901975,0.000000e+00,9.242925e-25,2.049321e-10,2.787971e-03,TRUE,3,330.8654,0.042887508,FALSE
65.5488551,2.450566e+02,FALSE,0.0233394320,0.02580643,0.023592174,7,FALSE,0.023592174,5.6385010,⋯,4.1737666,0.000000e+00,3.317621e-03,7.717481e-06,2.996046e-05,TRUE,3,365.7491,0.119781241,FALSE
12.1370401,8.537897e+01,FALSE,0.0000000100,0.05394949,0.015288009,8,FALSE,0.015288009,2.6700425,⋯,4.7811486,2.516350e-38,3.209101e-02,1.079980e-17,1.742965e-06,TRUE,5,231.7811,0.274464977,FALSE
634.8643569,1.939726e+04,FALSE,0.0216489997,0.02007163,0.021549783,7,FALSE,0.021549783,9.1499949,⋯,1.3119969,0.000000e+00,1.384914e-06,3.814237e-01,1.895212e-01,TRUE,2,550.2594,0.095178641,FALSE


## Find DE genes

In [13]:
res_strain_ypd <- results(dds, contrast = c("Group", "YPDH99",    "YPDmar1d"), tidy = TRUE)
res_strain_tc  <- results(dds, contrast = c("Group", "TCH99",     "TCmar1d"),  tidy = TRUE)
res_media_h99  <- results(dds, contrast = c("Group", "YPDH99",    "TCH99"),    tidy = TRUE)
res_media_mar  <- results(dds, contrast = c("Group", "YPDmar1d",  "TCmar1d"),  tidy = TRUE)

In [14]:
res_strain_ypd2 <- results(dds, contrast = c("Group", "YPDH99",    "YPDmar1d"), tidy = FALSE)
res_strain_tc2  <- results(dds, contrast = c("Group", "TCH99",     "TCmar1d"),  tidy = FALSE)
res_media_h992  <- results(dds, contrast = c("Group", "YPDH99",    "TCH99"),    tidy = FALSE)
res_media_mar2  <- results(dds, contrast = c("Group", "YPDmar1d",  "TCmar1d"),  tidy = FALSE)

In [15]:
cat("#### comparing h99 vs mar1d in YPD ####\n ")
summary(res_strain_ypd2)
cat("#### comparing h99 vs mar1d in TC ####\n")
summary(res_strain_tc2)
cat("#### comparing YPD -> TC in h99 ####\n")
summary(res_media_h992)
cat("#### comparing YPD -> TC  in mar1d ####\n")
summary(res_media_mar2)

#### comparing h99 vs mar1d in YPD ####
 
out of 8145 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2310, 28% 
LFC < 0 (down)   : 2675, 33% 
outliers [1]     : 0, 0% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

#### comparing h99 vs mar1d in TC ####

out of 8145 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2228, 27% 
LFC < 0 (down)   : 2342, 29% 
outliers [1]     : 0, 0% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

#### comparing YPD -> TC in h99 ####

out of 8145 with nonzero total read count
adjusted p-value < 0.1
LFC > 0 (up)     : 2551, 31% 
LFC < 0 (down)   : 3627, 45% 
outliers [1]     : 0, 0% 
low counts [2]   : 0, 0% 
(mean count < 0)
[1] see 'cooksCutoff' argument of ?results
[2] see 'independentFiltering' argument of ?results

#### compa

# Normalizing count to expression matrix

try different normalizing methods

In [16]:
rlgexp <- DESeq2::rlog(dds, blind = TRUE)
vstexp <- DESeq2::vst(dds,  blind = TRUE)
vomexp <- limma::voom(counts(dds))

store the expression matrix

In [17]:
gene_exp <- list()
gene_exp$cnt_unnorm <- counts(dds, normalize = FALSE)
gene_exp$cnt_norm   <- counts(dds, normalize = TRUE)
gene_exp$rlg        <- assay(rlgexp)
gene_exp$vst        <- assay(vstexp)
gene_exp$vom        <- vomexp$E

Print out the results

In [18]:
head(counts(dds, normalize = FALSE))

,A_1,A_13,A_14,A_15,A_2,A_3,B_16,B_26,B_27,B_38,⋯,G_21,G_22,G_23,G_9,H_12,H_24,H_35,H_36,H_45,H_47
CNAG_04548,0,0,0,0,2,0,0,0,0,1,⋯,0,2,0,0,0,1,1,4,1,0
CNAG_07304,1,1,3,3,1,3,17,2,7,15,⋯,43,29,23,21,20,25,3,19,13,17
CNAG_07305,0,0,0,0,0,0,1,0,0,1,⋯,5,1,1,2,3,3,0,1,2,1
CNAG_00002,10,21,18,23,14,12,92,11,98,173,⋯,139,135,59,55,55,105,18,86,44,64
CNAG_00003,3,4,7,8,5,3,44,8,26,45,⋯,76,64,27,39,60,58,9,61,26,32
CNAG_00004,9,37,31,57,12,18,215,22,108,299,⋯,370,308,115,114,192,239,44,187,119,189


In [19]:
head(counts(dds, normalize = TRUE))

,A_1,A_13,A_14,A_15,A_2,A_3,B_16,B_26,B_27,B_38,⋯,G_21,G_22,G_23,G_9,H_12,H_24,H_35,H_36,H_45,H_47
CNAG_04548,0.00000,0.00000,0.00000,0.00000,19.549476,0.00000,0.0000000,0.00000,0.000000,0.5641566,⋯,0.000000,0.9955391,0.000000,0.000000,0.000000,0.6126074,2.517925,2.4301194,1.454876,0.000000
CNAG_07304,11.80792,5.12013,21.62716,15.50888,9.774738,21.91315,13.5545290,10.26330,5.748723,8.4623484,⋯,17.996001,14.4353173,29.195382,15.742118,13.680974,15.3151848,7.553774,11.5430671,18.913391,17.085726
CNAG_07305,0.00000,0.00000,0.00000,0.00000,0.000000,0.00000,0.7973252,0.00000,0.000000,0.5641566,⋯,2.092558,0.4977696,1.269364,1.499249,2.052146,1.8378222,0.000000,0.6075298,2.909753,1.005043
CNAG_00002,118.07918,107.52273,129.76298,118.90145,136.846332,87.65259,73.3539219,56.44813,80.482119,97.5990843,⋯,58.173118,67.1988909,74.892502,41.229355,37.622679,64.3237762,45.322641,52.2475667,64.014555,64.322734
CNAG_00003,35.42375,20.48052,50.46338,41.35703,48.873690,21.91315,35.0823105,41.05319,21.352399,25.3870451,⋯,31.806885,31.8572520,34.272840,29.235361,41.042923,35.5312287,22.661321,37.0593206,37.826783,32.161367
CNAG_00004,106.27126,189.44480,223.48070,294.66881,117.296856,131.47888,171.4249261,112.89627,88.694580,168.6828105,⋯,154.849307,153.3130251,145.976911,85.457209,131.337352,146.4131667,110.788678,113.6080811,173.130274,189.953073


In [20]:
head(assay(rlgexp))

,A_1,A_13,A_14,A_15,A_2,A_3,B_16,B_26,B_27,B_38,⋯,G_21,G_22,G_23,G_9,H_12,H_24,H_35,H_36,H_45,H_47
CNAG_04548,-0.36041446,-0.456052853,-0.41689080,-0.45495167,0.07578108,-0.4153872,-0.64964672,-0.45579565,-0.6473709,-0.52488031,⋯,-0.6979455,-0.41804483,-0.61442089,-0.65439640,-0.6614310,-0.5119980,-0.30837535,-0.13521490,-0.3738135,-0.63189719
CNAG_07304,3.59919634,3.342069206,3.86919729,3.71955842,3.54679941,3.8749621,3.68154551,3.53905376,3.1735507,3.36662269,⋯,3.9046888,3.73137082,4.26964554,3.79161291,3.6893467,3.7738106,3.38060568,3.56977046,3.9107675,3.84700860
CNAG_07305,0.09794498,0.009903322,0.04595466,0.01091703,0.07803433,0.0473388,-0.01574098,0.01014009,-0.1467008,-0.06030272,⋯,0.1926520,-0.07514661,0.05033611,0.09060241,0.1714045,0.1446631,-0.06487865,-0.05118154,0.2313608,0.01648761
CNAG_00002,6.50785303,6.476198140,6.62714194,6.56598086,6.65211633,6.2906650,6.16184295,5.95890447,6.2445851,6.42272616,⋯,5.9587560,6.08459133,6.17926089,5.67907385,5.6065213,6.0464653,5.77940755,5.86899546,6.0440616,6.04723971
CNAG_00003,4.94473076,4.651337001,5.21430190,5.09033971,5.15830467,4.7038836,5.00777453,5.08527442,4.6124843,4.74225075,⋯,4.9277768,4.92865146,4.98386742,4.85652211,5.1448729,5.0205528,4.67927159,5.05689409,5.0637693,4.93355904
CNAG_00004,6.89611099,7.368716186,7.51202596,7.78738101,6.96261493,7.0491689,7.29118226,6.92385031,6.7055126,7.27680242,⋯,7.1974842,7.18810129,7.14196732,6.67382979,7.0466808,7.1455169,6.90122872,6.91728291,7.2983611,7.38699527


In [21]:
head(assay(vstexp))

,A_1,A_13,A_14,A_15,A_2,A_3,B_16,B_26,B_27,B_38,⋯,G_21,G_22,G_23,G_9,H_12,H_24,H_35,H_36,H_45,H_47
CNAG_04548,2.253742,2.253742,2.253742,2.253742,4.821897,2.253742,2.253742,2.253742,2.253742,2.747523,⋯,2.253742,2.907285,2.253742,2.253742,2.253742,2.768077,3.280224,3.262875,3.040776,2.253742
CNAG_07304,4.337817,3.688691,4.926842,4.591177,4.174183,4.940678,4.463487,4.215470,3.767306,4.055999,⋯,4.737790,4.522563,5.253133,4.605627,4.472132,4.579056,3.966950,4.317651,4.788086,4.685966
CNAG_07305,2.253742,2.253742,2.253742,2.253742,2.253742,2.253742,2.839597,2.253742,2.253742,2.747523,⋯,3.192714,2.717825,2.990021,3.052395,3.183908,3.135532,2.253742,2.765963,3.353787,2.910345
CNAG_00002,6.993664,6.868717,7.120371,7.002955,7.192087,6.599264,6.368366,6.036503,6.488115,6.740452,⋯,6.074118,6.256250,6.395073,5.652936,5.544539,6.200710,5.766681,5.940533,6.194607,6.200690
CNAG_00003,5.474125,4.869922,5.897730,5.656625,5.858479,4.940678,5.462869,5.647830,4.913417,5.098518,⋯,5.350059,5.351864,5.435820,5.254668,5.647532,5.477650,4.976230,5.526831,5.550900,5.362714
CNAG_00004,6.853158,7.635673,7.863699,8.248556,6.984767,7.138075,7.498550,6.933674,6.614722,7.476491,⋯,7.359742,7.346173,7.279554,6.566138,7.136623,7.283603,6.908545,6.942062,7.512101,7.639358


In [22]:
head(vomexp$E)

,A_1,A_13,A_14,A_15,A_2,A_3,B_16,B_26,B_27,B_38,⋯,G_21,G_22,G_23,G_9,H_12,H_24,H_35,H_36,H_45,H_47
CNAG_04548,-3.9356996,-3.6578558,-3.81319753,-3.6115822,-1.4657687,-3.7650195,-4.034228,-3.8773324,-3.6216742,-2.153254,⋯,-3.8426758,-1.314873,-2.929045,-3.505519,-3.5150854,-1.7321473,-0.3175950,-0.7375103,-0.9325319,-3.063366
CNAG_07304,-2.3507371,-2.0728933,-1.00584261,-0.8042273,-2.2027343,-0.9576646,1.095055,-1.5554043,0.2852164,1.215980,⋯,2.6002677,2.245842,2.625544,1.920745,1.8424666,2.3553155,0.9047974,1.3779670,2.2373931,2.065917
CNAG_07305,-3.9356996,-3.6578558,-3.81319753,-3.6115822,-3.7876968,-3.7650195,-2.449266,-3.8773324,-3.6216742,-2.153254,⋯,-0.3832442,-2.051839,-1.344083,-1.183591,-0.7077305,-0.5097549,-1.9025575,-2.3224728,-0.1955663,-1.478404
CNAG_00002,0.4566178,1.7684089,1.39625583,1.9430066,1.0702842,0.8788367,3.497153,0.6462296,4.0003776,4.700575,⋯,4.2814455,4.445348,3.965773,3.288897,3.2793304,4.4039894,3.3068958,3.5271930,3.9582390,3.947861
CNAG_00003,-1.1283447,-0.4879308,0.09369306,0.4758806,-0.3282652,-0.9576646,2.441505,0.2101305,2.1062462,2.769578,⋯,3.4147121,3.374426,2.852314,2.798261,3.4037778,3.5532549,2.3453700,3.0350793,3.2104260,2.959002
CNAG_00004,0.3122279,2.5709629,2.16408239,3.2339078,0.8561594,1.4444338,4.717316,1.6145207,4.1398770,5.488196,⋯,5.6906539,5.632325,4.922704,4.333685,5.0736292,5.5867720,4.5731759,4.6433115,5.3833724,5.502688


# Store the results

In [25]:
#outfile <- file.path(OUTDIR, "HTS-GroupB-DESeq.RData")
outfile <- file.path(OUTDIR, "HTS-GroupAll-DESeq.RData")

save(countData, columnData, countData_rm0F, columnData_rmF,
     
     res_media_h99,   res_media_mar,  res_strain_ypd,  res_strain_tc,
     res_media_h992,  res_media_mar2, res_strain_ypd2, res_strain_tc2,
     
     dds, rlgexp, vstexp, vomexp, gene_exp, 
     
     file = outfile)

tools::md5sum(outfile)

/shared_space/TA_clint/analysis_output/out/HTS-GroupAll-DESeq.RData 
                                 "ae1f353c2680e1aa378bfc2968bad0cc"

# The end

In [24]:
sessionInfo()

R version 3.4.1 (2017-06-30)
Platform: x86_64-pc-linux-gnu (64-bit)
Running under: Debian GNU/Linux 8 (jessie)

Matrix products: default
BLAS: /opt/conda/lib/R/lib/libRblas.so
LAPACK: /opt/conda/lib/R/lib/libRlapack.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
 [1] tools     parallel  stats4    stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] bindrcpp_0.2               plotly_4.8.0              
 [3] dendextend_1.8.0           gridExtra_2.3             
 [5] RColorBrewer_1.1-2         qvalue_2.10.0             
 [7] limma_3.34.9               DESeq2_1.18.1             
 [9] SummarizedExperi